In [1]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
import torch
import torch.optim as optim
import torch.nn.functional as F
from torch import nn
from torch.utils.data import TensorDataset, DataLoader

In [6]:
# load data from csv file
train_data = pd.read_csv('train_data.csv', header=None)
train_targets = pd.read_csv('train_target.csv', header=None)
test_data = pd.read_csv('test_data.csv', header=None)

# convert data to numpy arrays
X_train = train_data.values.reshape(-1, 1, 48, 48).astype(np.float32)
y_train = train_targets.values.squeeze().astype(np.int64)
X_test = test_data.values.reshape(-1, 1, 48, 48).astype(np.float32)

# normalize pixel values from [0, 255] to [0, 1]
X_train /= 255.0
X_test /= 255.0

In [7]:
# ensuring data is in the correct shape
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)

(16175, 1, 48, 48)
(16175,)
(3965, 1, 48, 48)


In [8]:
# convert y_train to one-hot encoding
y_train_one_hot = F.one_hot(torch.tensor(y_train), num_classes=3).float()

# convert all data to PyTorch Tensors
X_train_tensor = torch.tensor(X_train)
y_train_tensor = y_train_one_hot
X_test_tensor = torch.tensor(X_test)

# checking tensor types for debugging
print(f'X_train_tensor type: {type(X_train_tensor)}, shape: {X_train_tensor.shape}')
print(f'y_train_tensor type: {type(y_train_tensor)}, shape: {y_train_tensor.shape}')
print(f'X_test_tensor type: {type(X_test_tensor)}, shape: {X_test_tensor.shape}')

# creating tensor datasets and dataloaders
train_tensor = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_tensor, batch_size=64, shuffle=True)

test_tensor = TensorDataset(X_test_tensor)
test_loader = DataLoader(test_tensor, batch_size=64, shuffle=False)

X_train_tensor type: <class 'torch.Tensor'>, shape: torch.Size([16175, 1, 48, 48])
y_train_tensor type: <class 'torch.Tensor'>, shape: torch.Size([16175, 3])
X_test_tensor type: <class 'torch.Tensor'>, shape: torch.Size([3965, 1, 48, 48])


In [5]:
print(X_train.shape[0])

16175


In [9]:
# defining the model
model = nn.Sequential(
    nn.Conv2d(1, 32, kernel_size=3, padding=1),
    nn.BatchNorm2d(32),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=2, stride=2),
    nn.Dropout(0.25),

    nn.Conv2d(32, 64, kernel_size=3, padding=1),
    nn.BatchNorm2d(64),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=2, stride=2),
    nn.Dropout(0.25),

    nn.Conv2d(64, 128, kernel_size=3, padding=1),
    nn.BatchNorm2d(128),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=2, stride=2),
    nn.Dropout(0.25),

    nn.Flatten(),  # Flatten the tensor before fully connected layers
    nn.Linear(128 * 6 * 6, 256), # 128*6*6 = 4608, the output size after the final pooling layer
    nn.BatchNorm1d(256),
    nn.ReLU(),
    nn.Dropout(0.5),

    nn.Linear(256, 128),
    nn.BatchNorm1d(128),
    nn.ReLU(),
    nn.Dropout(0.5),

    nn.Linear(128, 64),
    nn.BatchNorm1d(64),
    nn.ReLU(),

    nn.Linear(64, 32),
    nn.BatchNorm1d(32),
    nn.ReLU(),

    nn.Linear(32, 16),
    nn.BatchNorm1d(16),
    nn.ReLU(),

    nn.Linear(16, 3)
)

In [10]:
print(model)

Sequential(
  (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU()
  (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (4): Dropout(p=0.25, inplace=False)
  (5): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (6): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (7): ReLU()
  (8): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (9): Dropout(p=0.25, inplace=False)
  (10): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (12): ReLU()
  (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (14): Dropout(p=0.25, inplace=False)
  (15): Flatten(start_dim=1, end_dim=-1)
  (16): Linear(in_features=4608, out_features=256, bias=Tru

In [9]:
# define loss function, optimizer, and learning rate scheduler
'''
optimizer uses weight decay to penalize large weights
scheduler reduces learning rate when validation loss plateaus
'''
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=5, verbose=True)

/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


In [11]:
# train the model with early stopping if model converges
num_epochs = 50
patience = 5
best_loss = float('inf')
patience_counter = 0

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = loss_fn(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    epoch_loss = running_loss / len(train_loader)
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}')
    scheduler.step(epoch_loss)

    # Early stopping
    if epoch_loss < best_loss:
        best_loss = epoch_loss
        patience_counter = 0
    else:
        patience_counter += 1
        if patience_counter >= patience:
            print('Early stopping triggered')
            break


# predictions on test data
model.eval()
predictions = []
with torch.no_grad():
    for images in test_loader:
        images = images[0].to(torch.float32)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        predictions.extend(predicted.numpy())

# submission file
submission = pd.DataFrame({'Id': np.arange(len(predictions)), 'Category': predictions})
submission.to_csv('submission.csv', index=False)

Epoch [1/50], Loss: 0.8006
Epoch [2/50], Loss: 0.7268
Epoch [3/50], Loss: 0.6787
Epoch [4/50], Loss: 0.6527
Epoch [5/50], Loss: 0.6198
Epoch [6/50], Loss: 0.6031
Epoch [7/50], Loss: 0.5892
Epoch [8/50], Loss: 0.5695
Epoch [9/50], Loss: 0.5508
Epoch [10/50], Loss: 0.5370
Epoch [11/50], Loss: 0.5254
Epoch [12/50], Loss: 0.5152
Epoch [13/50], Loss: 0.4930
Epoch [14/50], Loss: 0.4877
Epoch [15/50], Loss: 0.4751
Epoch [16/50], Loss: 0.4512
Epoch [17/50], Loss: 0.4483
Epoch [18/50], Loss: 0.4383
Epoch [19/50], Loss: 0.4211
Epoch [20/50], Loss: 0.4135
Epoch [21/50], Loss: 0.4011
Epoch [22/50], Loss: 0.3855
Epoch [23/50], Loss: 0.3768
Epoch [24/50], Loss: 0.3654
Epoch [25/50], Loss: 0.3564
Epoch [26/50], Loss: 0.3525
Epoch [27/50], Loss: 0.3348
Epoch [28/50], Loss: 0.3354
Epoch [29/50], Loss: 0.3224
Epoch [30/50], Loss: 0.3173
Epoch [31/50], Loss: 0.3139
Epoch [32/50], Loss: 0.3043
Epoch [33/50], Loss: 0.2978
Epoch [34/50], Loss: 0.2855
Epoch [35/50], Loss: 0.2857
Epoch [36/50], Loss: 0.2803
E

In [8]:
# predictions on test data
model.eval()
predictions = []
with torch.no_grad():
    for images in test_loader:
        images = images[0].to(torch.float32)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        predictions.extend(predicted.numpy())

# submission file
submission = pd.DataFrame({'Id': np.arange(len(predictions)), 'Category': predictions})
submission.to_csv('submission.csv', index=False)